In [43]:
import os
import openai
import shutil

In [44]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [45]:
!pip install Gitpython

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [46]:
from git import Repo

In [47]:
from pathlib import Path

In [48]:
#Just testing printing working directory

In [49]:
pwd

'C:\\Users\\david\\Desktop\\Python\\Courses\\Udemy\\OpenAI Python Bootcamp\\dawidahs.github.io'

In [50]:
PATH_TO_BLOG_REPO = Path('C:\\Users\\david\\Desktop\\Python\\Courses\\Udemy\\OpenAI Python Bootcamp\\dawidahs.github.io\\.git')

In [51]:
PATH_TO_BLOG = PATH_TO_BLOG_REPO.parent

In [52]:
PATH_TO_CONTENT = PATH_TO_BLOG/'content'

In [53]:
PATH_TO_CONTENT

WindowsPath('C:/Users/david/Desktop/Python/Courses/Udemy/OpenAI Python Bootcamp/dawidahs.github.io/content')

In [54]:
PATH_TO_CONTENT.mkdir(exist_ok=True,parents=True)

In [55]:
def update_blog(commit_message="Updates blog"):
    repo = Repo(PATH_TO_BLOG_REPO)
    repo.git.add(all=True)
    repo.index.commit(commit_message)
    origin = repo.remote(name='origin')
    origin.push()

In [56]:
#random_text_string = 'smlkmssdlsdvdsdvsdmsdl'

In [57]:
#with open(PATH_TO_BLOG/'index.html', 'w') as f:
    #f.write(random_text_string)

In [58]:
 #Test push an update

In [59]:
#update_blog()

In [60]:
def create_new_blog(title, content, cover_image):
    cover_image = Path(cover_image)
    
    files = len(list(PATH_TO_CONTENT.glob('*.html')))
    new_title = f'{files+1}.html'
    path_to_new_content = PATH_TO_CONTENT/new_title
    
    shutil.copy(cover_image,PATH_TO_CONTENT)
    
    if not os.path.exists(path_to_new_content):
        with open(path_to_new_content, 'w') as f:
            f.write('<DOCTYPE HTML>\n')
            f.write('<html>\n')
            f.write('<head>\n')
            f.write(f'<title>{title}</title>\n')
            f.write('</head>\n')
            
            f.write('<body>\n')
            f.write(f"<image src='{cover_image.name}' alt='Cover Image'>\n")
            f.write(f'<h1>{title}</h1>\n')
            f.write(content.replace("\n", "<br />\n"))
            f.write('</body>\n')
            
            f.write('</html>\n')
            print('Blog Created')
            return(path_to_new_content)
    else:
        raise FileExistsError('File already exists, please check you name.')

In [61]:
path_to_new_content = create_new_blog('Test Title','aklfakssadansdlasn','test.jpg')

Blog Created


In [68]:
from bs4 import BeautifulSoup as Soup

In [71]:
with open(PATH_TO_BLOG/'index.html') as index:
    soup = Soup(index.read())

In [72]:
str(soup)

'<html lang="en">\n<head>\n<meta charset="utf-8"/>\n<meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n<meta content="width=device-width, initial-scale=1.0" name="viewport"/>\n<title>Document</title>\n</head>\n<body>\n<h1>My Blog Home Page</h1>\n<a href="index.html">Home</a>\n</body>\n</html>'

In [73]:
def check_for_duplicate_links(path_to_new_content, links):
    urls = [str(link.get('href'))for link in links]
    content_path = str(Path(*path_to_new_content.parts[-2:]))
    return content_path in urls

In [74]:
def write_to_index(path_to_new_content):
    with open(PATH_TO_BLOG/'index.html') as index:
        soup = Soup(index.read())
        
    links = soup.find_all('a')
    last_link = links[-1]
    
    if check_for_duplicate_links(path_to_new_content, links):
        raise ValueError('Link already exist!')
        
        link_to_new_blog = soup.new_tag('a', href=Path(*path_to_new_content.parts[-2:]))
        link_to_new_blog_string = path_to_new_content.name.split('.')[0]
        last_link.insert_after(link_to_new_blog)
        
        with open(PATH_TO_BLOG/'index.html', 'w') as f:
            f.write(str(soup.prettefy(fortmatter="html")))

In [76]:
write_to_index(path_to_new_content)

NameError: name 'content_pat' is not defined

In [ ]:
update_blog()